Imports

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Read Data

In [41]:
file = pd.ExcelFile("../data/v4/Handball 3. Liga vom 19.12.2022.xlsx")

file.sheet_names

df = file.parse('3. Liga')

# drop last col comments
df = df.iloc[:, :-1]
df.columns

Index(['Heimmannschaft', 'Gastmannschaft', 'Spieltag', 'Name', 'Spielminute',
       'Differenz Pfiff 1 und 2 (in Sekunden)', 'Tore Verein', 'Tore Gegner',
       'Torwartposition (1 = vorne, 2 = hinten)',
       'Wurf erfolgreich (0= Nein, 1 = Ja, 2 = im Nachwurf)',
       'L-R-Händer (1= Rechtshänder; 2= Linkshänder)', 'Wurfrichtung (1-9)?',
       'Drucksituation?\n0 = Nein, 1 = Torwartwechsel\n2 = letzter 7 Meter gehalten\n3 = Tordifferenz =< 4 + ab 50. Spielminute\n4 = 1 + 2, 5 = 1 + 3, 6 = 2 + 3, 7 = 1 + 2 + 3, 8 = letzer 7-Meter erst im Nachwurf erfolgreich',
       'Anzahl 7-Meter im Spiel (der wievielte 7-Meter im Spiel vom Spieler)',
       'Wurfquadrant des letzten 7-Meters (1-9)',
       'letzter 7-Meter erfolgreich \n(1= ja; 0= Nein; 2 = im Nachwurf erfolgreich)?'],
      dtype='object')

Cleanup & Renaming

In [42]:
df.replace('unklar', np.nan,inplace=True)
df.replace('unklar ', np.nan,inplace=True)
df.replace('fehlend', np.nan,inplace=True)
#df.dropna(inplace=True)
df.replace('-', np.nan,inplace=True)
df.columns =[ 'homeTeam','opponentTeam','day','name','playTime',
              'whistleThrowTime','homeGoals','opponentGoals','goalKeeperPosition',
              'throwSuccess','LRHandy','throwDirection','pressureSituation','noOfThrowsFromPlayer',
              'throwQuadrantLastThrow','lastThrowSuccess']
df = df.astype({'homeTeam':'string', 'opponentTeam':'string','day':'float16','name':'string','playTime':'float16',
              'homeGoals':'float16','opponentGoals':'float16','whistleThrowTime':'float16','goalKeeperPosition':'float16',
              'throwSuccess':'float16','LRHandy':'float16','throwDirection':'float16','pressureSituation':'float16','noOfThrowsFromPlayer':'float16',
              'throwQuadrantLastThrow':'float16','lastThrowSuccess':'float16' })

General Summary

In [43]:
ps = df.loc[:,('throwSuccess','pressureSituation')].copy()
ps.dropna(inplace=True)
ps['throwSuccess'].replace(2.0, 1.0,inplace=True)
s = [0, 1,4,5,7]
ps = ps[ps.pressureSituation.isin(s) == True]
ps['pressureSituation'].replace(4.0, 1.0,inplace=True)
ps['pressureSituation'].replace(5.0, 1.0,inplace=True)
ps['pressureSituation'].replace(7.0, 1.0,inplace=True)
# print(ps)
# X_ps = ['No Pressure Situation','GoalKeeperChange','LastThrowFailed',
#      'GoalDiffLessEqualTo4After50MinPlay','GoalKeeperChange-LastThrowFailed',
#      'GoalKeeperChange-GoalDiffLessEqualTo4After50MinPlay','LastThrowFailed-GoalDiffLessEqualTo4After50MinPlay',
#      'GoalKeeperChange-LastThrowFailed-GoalDiffLessEqualTo4After50MinPlay',
#      'FirstInNachWurf']
# for i,s in enumerate(X_ps):
#     ps['pressureSituation'].replace(i, s,inplace=True)

# print(ps)
# ps1 =pd.DataFrame(ps.groupby('throwSuccess').pressureSituation.value_counts()).unstack()
# ps1
contigency= pd.crosstab(ps['throwSuccess'], ps['pressureSituation']) 
contigency



pressureSituation,0.0,1.0
throwSuccess,,
0.0,39,5
1.0,129,37


In [44]:
contigency_pct = pd.crosstab(ps['throwSuccess'], ps['pressureSituation'], normalize='index')
contigency_pct

pressureSituation,0.0,1.0
throwSuccess,,
0.0,0.886364,0.113636
1.0,0.777108,0.222892


In [45]:
from scipy.stats import chi2_contingency


# null hypo : no significant association between pressure situation and throwsuccess
# research hypo : There is relation between pressure situation and throwsuccess
# for df=1 & alpha=0.05 c = 3.84 (Chi-square table)
c, p, dof, expected = chi2_contingency(contigency) 
print(c,p,dof)
print(expected)
#c > 3.84  Null Hypo accepted with 5% error

# Also,
# P-value ≤ α: The variables have a statistically significant association (Reject H0)
# If the p-value is less than or equal to the significance level, you reject the null hypothesis and conclude that there is a statistically significant association between the variables.
# P-value > α: Cannot conclude that the variables are associated (Fail to reject H0)
# If the p-value is larger than the significance level, you fail to reject the null hypothesis because there is not enough evidence to conclude that the variables are associated.

1.9568900602409636 0.1618463117995769 1
[[ 35.2   8.8]
 [132.8  33.2]]
